## Uso YT dlp para la demo, scrapping basico de metadatos. Quiza tengo que pasar a una api oficial para que sea mas estable o tener acceso a mas datos

In [8]:
import yt_dlp
import json
import time

lista_canales = [
    "https://www.youtube.com/@BabyBusES",
    "https://www.youtube.com/@SmileandLearnEspa%C3%B1ol",
    "https://www.youtube.com/@creativoskids",
    "https://www.youtube.com/@JuguetesyColores",
    "https://www.youtube.com/@GenevievesPlayhouse",
    "https://www.youtube.com/@GamingWithGTA00",
    "https://www.youtube.com/@videosdejuguetespawpatrole5118",
]

def scrapear_lista_canales(urls):
    ydl_opts = {
        'quiet': True,
        'extract_flat': True,  # (solo metadatos)
        'playlistend': 10,     # Límite de videos
        'ignoreerrors': True,  
        'no_warnings': True,
        # 'sleep_interval': 1,
    }

    resultados = []

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        
        for index, url in enumerate(urls):
            
            try:
                # Extraemos la info
                info = ydl.extract_info(url, download=False)
                
                # A veces yt-dlp devuelve None si la url está mal
                if not info:
                    print(f" No se pudo obtener info de {url}")
                    continue

                # Estructuramos los datos
                datos = {
                    "id_canal": info.get('id'),
                    "nombre": info.get('uploader') or info.get('title'),
                    "descripcion": info.get('description', ''),
                    "suscriptores": info.get('channel_follower_count'),
                    "url_origen": url,
                    # Juntamos los títulos de los videos en una lista simple
                    "videos_recientes": []
                }

                # Extraer videos (validando que existan entradas)
                if 'entries' in info:
                    for video in info['entries']:
                        if video and video.get('title'):
                            datos["videos_recientes"].append(video.get('title'))
                
                resultados.append(datos)
                print(f"    OK: {datos['nombre']} ({len(datos['videos_recientes'])} videos)")

            except Exception as e:
                print(f"    Error en este canal: {str(e)}")

    return resultados

# --- EJECUCIÓN ---

start = time.time()
data_scraped = scrapear_lista_canales(lista_canales)
end = time.time()

print(f"\n✨ Proceso terminado en {end - start:.2f} segundos.")

# 2. Guardar en un archivo JSON (Esto es lo que usará tu IA después)
nombre_archivo = "dataset_canales.json"
with open(nombre_archivo, 'w', encoding='utf-8') as f:
    json.dump(data_scraped, f, ensure_ascii=False, indent=4)

print(f"📂 Datos guardados en '{nombre_archivo}'.")

    OK: BabyBus - Canciones Infantiles & Videos para Niños (3 videos)
    OK: Smile and Learn - Español (3 videos)
    OK: Creativos Kids (2 videos)
    OK: Juguetes y Colores (10 videos)
    OK: Genevieve's Playhouse - Learning Videos for Kids (2 videos)
    OK: Gaming With GTA  (10 videos)
    OK: Videos de juguetes Paw Patrol en español (10 videos)

✨ Proceso terminado en 5.07 segundos.
📂 Datos guardados en 'dataset_canales.json'.


In [9]:
import json
import time
from sentence_transformers import SentenceTransformer, util

try:
    with open('dataset_canales.json', 'r', encoding='utf-8') as f:
        datos_canales = json.load(f)
    print(f"   --> {len(datos_canales)} canales cargados.")
except FileNotFoundError:
    print(" Error: No encontré el archivo 'dataset_canales.json'. Ejecuta el scraper primero.")
    exit()

# El modelo necesita un solo texto por canal para entender de qué trata.
corpus_textos = []

for canal in datos_canales:
    # Unimos los titulos de videos con comas
    txt_videos = ", ".join(canal.get('videos_recientes', []))
    
    # Creamos el texto maestro para la IA
    # "Nombre: [X]. Descripción: [Y]. Contenido reciente: [Z]"
    texto_full = f"Canal: {canal['nombre']}. Descripción: {canal['descripcion']}. Temas de videos: {txt_videos}"
    
    corpus_textos.append(texto_full)

# --- INDEXACIÓN  ---
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

embeddings_db = model.encode(corpus_textos, convert_to_tensor=True)

def buscar(query_usuario, top_k=3):
    start_time = time.time()
    
    # Vectorizamos lo que escribió el usuario
    query_embedding = model.encode(query_usuario, convert_to_tensor=True)
    
    # Buscamos similitud
    hits = util.semantic_search(query_embedding, embeddings_db, top_k=top_k)
    
    print(f" Búsqueda: '{query_usuario}' ({time.time() - start_time:.4f} seg)")
    
    for hit in hits[0]:
        score = hit['score']
        id_canal = hit['corpus_id'] # El índice en la lista original
        canal_original = datos_canales[id_canal] # Recuperamos el objeto JSON original
        
        # Filtro de calidad
        if score < 0.25: 
            continue
            
        print(f" Match: {score:.2f} | {canal_original['nombre']}")
        # por que hizo match?
        print(f"      (Contexto: {canal_original['videos_recientes'][:2]}...)")
    print("-" * 30)

# --- PRUEBAS ---
# Ahora simulamos las queries libres de tus clientes
buscar("videos")
buscar("aprender")
buscar("juegos")
buscar("tutoriales")
buscar("dibujos animados para bebes")
buscar("niños creativos")
buscar("diversion")
buscar("jugar")

   --> 7 canales cargados.
 Búsqueda: 'videos' (0.0247 seg)
 Match: 0.52 | Videos de juguetes Paw Patrol en español
      (Contexto: ['Orbeez Pool Games! 🎉✨ | Daycare', 'Chase from Paw Patrol and Roy from Robocars pick up the trash 🗑️✨ | Daycare']...)
 Match: 0.49 | Genevieve's Playhouse - Learning Videos for Kids
      (Contexto: ["Genevieve's Playhouse - Learning Videos for Kids - Videos", "Genevieve's Playhouse - Learning Videos for Kids - Shorts"]...)
 Match: 0.46 | Creativos Kids
      (Contexto: ['Creativos Kids - Videos', 'Creativos Kids - Shorts']...)
------------------------------
 Búsqueda: 'aprender' (0.0189 seg)
 Match: 0.48 | Smile and Learn - Español
      (Contexto: ['Smile and Learn - Español - Videos', 'Smile and Learn - Español - Live']...)
 Match: 0.43 | Genevieve's Playhouse - Learning Videos for Kids
      (Contexto: ["Genevieve's Playhouse - Learning Videos for Kids - Videos", "Genevieve's Playhouse - Learning Videos for Kids - Shorts"]...)
 Match: 0.36 | Creativo